In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# New version 

In [3]:
def get_landmarks(image_rgb, pose_model): 
    '''
    Récupère les landmarks d'une image donnée en RGB via un modèle déjà chargé.
    Return: un numpy array (33,4) ou None si rien n'est trouvé.
    '''

    results = pose_model.process(image_rgb)
    
    if results.pose_landmarks:

        pose_np = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark])
        return pose_np

    return None

def mmss(seconds: float) -> str:
    s = max(0, int(seconds))
    return f"{s//60:02d}:{s%60:02d}"

def draw_rounded_rect(img, pt1, pt2, radius=22, color=(255, 200, 230), thickness=-1):
    x1, y1 = pt1
    x2, y2 = pt2
    radius = int(max(0, min(radius, abs(x2-x1)//2, abs(y2-y1)//2)))
    if thickness < 0:
        cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, -1)
        cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, -1)
        cv2.circle(img, (x1+radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y1+radius), radius, color, -1)
        cv2.circle(img, (x1+radius, y2-radius), radius, color, -1)
        cv2.circle(img, (x2-radius, y2-radius), radius, color, -1)
    else:
        cv2.rectangle(img, pt1, pt2, color, thickness)

def overlay_alpha(dst, overlay, alpha=0.75):
    cv2.addWeighted(overlay, alpha, dst, 1-alpha, 0, dst)

def put_fit_text(img, text, org, max_width, font=cv2.FONT_HERSHEY_SIMPLEX, base_scale=1.0, thickness=2, color=(60,30,60)):
    (tw, th), _ = cv2.getTextSize(text, font, base_scale, thickness)
    scale = base_scale if tw <= max_width else base_scale * (max_width / tw)
    cv2.putText(img, text, org, font, scale, color, thickness, cv2.LINE_AA)

def draw_session_card(frame, session, idx, paused=False, phase="HOLD"):
    x, y = 20, 20
    box_w, box_h = 520, 150
    draw_rounded_rect(frame, (x, y), (x+box_w, y+box_h), radius=20, color=(255, 245, 250), thickness=-1)
    cv2.putText(frame, "YOGA SESSION", (x+20, y+35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (60,30,60), 2)
    cv2.putText(frame, f"{idx+1}/{len(session)}", (x+box_w-70, y+35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (60,30,60), 2)
    posture_name = session[idx]["name"].upper()
    put_fit_text(frame, posture_name, (x+20, y+85), max_width=box_w-40, base_scale=1.2)
    status_text = "PAUSED" if paused else f"STATUS: {phase}"
    cv2.putText(frame, status_text, (x+20, y+130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (120,40,120), 2)

def draw_progress_bar_bottom(frame, progress, right_text=""):
    H, W = frame.shape[:2]
    margin, bar_h = 25, 20
    x, y, w = margin, H - 45, W - 2 * margin
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (240, 230, 240), -1) # BG
    cv2.rectangle(frame, (x, y), (x+int(w*progress), y+bar_h), (255, 180, 220), -1) # Fill
    cv2.rectangle(frame, (x, y), (x+w, y+bar_h), (60, 30, 60), 1) # Border
    if right_text:
        cv2.putText(frame, right_text, (x+w-60, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (60,30,60), 2)

In [5]:
# Ordre des sorties du modèle KERAS : 0=Cobra, 1=Tree, 2=Dog 
labels = ['Cobra', 'Tree', 'Dog']
# model = tf.keras.models.load_model('modele.h5')

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Session de Yoga
session = [
    {"name": "Cobra", "duration": 15},
    {"name": "Tree", "duration": 20},
    {"name": "Dog", "duration": 10}
]

#Resize la caméra 
cap = cv2.VideoCapture(0)
cam_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cam_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
zoom = 1.2
cv2.namedWindow("Yoga postures", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Yoga postures", int(cam_w*zoom), int(cam_h*zoom))

#initialization of variables
idx = 0
paused = False
start_time = None
elapsed_time = 0


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_model:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        landmarks_np = get_landmarks(image_rgb, pose_model)
        image_rgb.flags.writeable = True
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        

        # # Modele prediction posture 
        # is_pose_correct = False
        
        # if landmarks_np is not None:
        #     input_data = landmarks_np.flatten().reshape(1, -1) #format (33,4) --> (1, 132)
            
        #     prediction = model.predict(input_data, verbose=0)
        #     predicted_idx = np.argmax(prediction)
        #     confidence = prediction[0][predicted_idx]
            
        #     current_target_name = session[idx]["name"]
        #     predicted_name = labels[predicted_idx]
            
        #     if predicted_name == current_target_name and confidence > 0.8:
        #         is_pose_correct = True

        is_pose_correct = landmarks_np is not None 

        phase = "WAITING..."
        progress = 0
        
        if is_pose_correct and not paused:
            phase = "HOLDING"
            if start_time is None:
                start_time = time.time() - elapsed_time
            elapsed_time = time.time() - start_time
            
            target_duration = session[idx]["duration"]
            progress = np.clip(elapsed_time / target_duration, 0, 1)

            if elapsed_time >= target_duration:
                if idx < len(session) - 1:
                    idx += 1
                    start_time = None
                    elapsed_time = 0
                else:
                    phase = "SESSION DONE!"
        else:
            start_time = None 

        
        # if results and results.pose_landmarks:
        #     mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        draw_session_card(image_bgr, session, idx, paused=paused, phase=phase)
        
        remaining = session[idx]["duration"] - elapsed_time
        draw_progress_bar_bottom(image_bgr, progress, right_text=mmss(remaining))

        cv2.putText(image_bgr, "p:Pause  n:Next  q:Quit", (30, image_bgr.shape[0]-70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (100,40,100), 2)

        cv2.imshow('Yoga postures', image_bgr)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        elif key == ord('p'): paused = not paused
        elif key == ord('n'): # Skip to next
            if idx < len(session)-1: 
                idx += 1
                elapsed_time = 0
                start_time = None

    cap.release()
    cv2.destroyAllWindows()